<a href="https://colab.research.google.com/github/kotlanandu525/Stacking_EnsembleLearning/blob/main/Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image.png](
  
)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('/content/Social_Network_Ads.csv')
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [2]:
df.isnull().sum()

,0
User ID,0
Gender,0
Age,0
EstimatedSalary,0
Purchased,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier


In [5]:
x=df[['Age','EstimatedSalary']]
y=df['Purchased']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
#

# Define Base Models


In [8]:
base_models={
    ('lr',LogisticRegression()),
    ('dt',DecisionTreeClassifier(max_depth=3)),
    ('knn',KNeighborsClassifier(n_neighbors=5))
}


# Define Meta Model
Meta model defines how to combine predictions


In [25]:
meta_model=DecisionTreeClassifier(max_depth=3)


# **Build Stacking Classifier**

In [26]:
classifier=StackingClassifier(
    estimators=list(base_models),
    final_estimator=meta_model,
    cv=5
)
classifier.fit(x_train,y_train)

StackingClassifier(cv=5,
                   estimators=[('dt', DecisionTreeClassifier(max_depth=3)),
                               ('knn', KNeighborsClassifier()),
                               ('lr', LogisticRegression())],
                   final_estimator=DecisionTreeClassifier(max_depth=3))

In [27]:
y_pred=classifier.predict(x_test)

In [28]:

from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_auc_score
print(accuracy_score(y_test,y_pred))

0.9


In [29]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92        52
           1       0.81      0.93      0.87        28

    accuracy                           0.90        80
   macro avg       0.89      0.91      0.89        80
weighted avg       0.91      0.90      0.90        80

[[46  6]
 [ 2 26]]


**Practice
**

In [34]:
df=pd.read_csv('/content/kc_house_data.csv.zip')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [35]:
df.isnull().sum()

,0
id,0
date,0
price,0
bedrooms,0
bathrooms,0
sqft_living,0
sqft_lot,0
floors,0
waterfront,0
view,0


In [37]:
df.drop(['id','date'],axis=1,inplace=True)

In [38]:
x=df.drop('price',axis=1)
y=df['price']

In [39]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [40]:
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)


In [49]:
from sklearn.linear_model import LinearRegression
meta_model=LinearRegression()

In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
base_models=[
    ('lr',LinearRegression()),
    ('rf',RandomForestRegressor(n_estimators=150,
        max_depth=10,
        random_state=42)),
    ('knn',KNeighborsRegressor(
        n_neighbors=5
    ))
]

In [51]:
from sklearn.ensemble import StackingRegressor
classifier=StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
    )
classifier.fit(x_train,y_train)

StackingRegressor(cv=5,
                  estimators=[('lr', LinearRegression()),
                              ('rf',
                               RandomForestRegressor(max_depth=10,
                                                     n_estimators=150,
                                                     random_state=42)),
                              ('knn', KNeighborsRegressor())],
                  final_estimator=LinearRegression())

In [53]:
y_pred=classifier.predict(x_test)

In [61]:
from sklearn.metrics import *
print(r2_score(y_test,y_pred))

0.8453336512432479


In [62]:
print(mean_squared_error(y_test,y_pred))

23381950101.81815


# classifier practice

In [105]:
df=pd.read_csv('/content/train_u6lujuX_CVtuZ9i.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [106]:
df.isnull().sum()

,0
Loan_ID,0
Gender,13
Married,3
Dependents,15
Education,0
Self_Employed,32
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,22
Loan_Amount_Term,14


In [107]:
cat_cols=df.select_dtypes(include='object').columns
num_cols=df.select_dtypes(include='number').columns
#

In [108]:
for col in cat_cols:
  df[col].fillna(df[col].mode()[0],inplace=True)
#

/tmp/ipython-input-2827958460.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0],inplace=True)


In [109]:
for col in num_cols:
  df[col].fillna(df[col].median(),inplace=True)

/tmp/ipython-input-3649106418.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(),inplace=True)


In [96]:
from sklearn.preprocessing import LabelEncoder

#

In [110]:
df.isnull().sum()

,0
Loan_ID,0
Gender,0
Married,0
Dependents,0
Education,0
Self_Employed,0
ApplicantIncome,0
CoapplicantIncome,0
LoanAmount,0
Loan_Amount_Term,0


In [111]:
df['Loan_Status']

,Loan_Status
0,Y
1,N
2,Y
3,Y
4,Y
...,...
609,Y
610,Y
611,Y
612,Y


In [121]:
df['Self_Employed']=LabelEncoder().fit_transform(df['Self_Employed'])

In [132]:
x=df[['Self_Employed','ApplicantIncome','LoanAmount','CoapplicantIncome']]
y=df['Loan_Status'].map({'Y':1,'N':0})
#

In [133]:

#

In [140]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
#

In [150]:
base_models=[
    ('lr',LogisticRegression()),
    ('dt',DecisionTreeClassifier(max_depth=3)),
    ('knn',KNeighborsClassifier(n_neighbors=5))
]

In [163]:
meta_model=LogisticRegression()

In [164]:
classifier=StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)
classifier.fit(x_train,y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression()),
                               ('dt', DecisionTreeClassifier(max_depth=3)),
                               ('knn', KNeighborsClassifier())],
                   final_estimator=LogisticRegression())

In [165]:
y_pred=classifier.predict(x_test)


In [166]:
accuracy_score(y_test,y_pred)

0.6504065040650406